In [36]:
import pandas as pd
import numpy as np
import scipy.sparse as sps

from scipy.sparse import *

In [37]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

In [38]:
urm_path = './content/data_train.csv'
urm_all_df = pd.read_csv(filepath_or_buffer=urm_path,
                                sep=",",
                                header=0,
                                dtype={0:int, 1:int, 2:float},
                                engine='python')

urm_all_df.columns = ["UserID", "ItemID", "Interaction"]

In [39]:
userID_unique = urm_all_df["UserID"].unique()
itemID_unique = urm_all_df["ItemID"].unique()

n_users = len(userID_unique)
n_items = len(itemID_unique)
n_interactions = len(urm_all_df)

In [40]:
urm_all = sps.coo_matrix((urm_all_df["Interaction"].values,
                          (urm_all_df["UserID"].values, urm_all_df["ItemID"].values)))

urm_all

<13025x22348 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in COOrdinate format>

In [41]:
URM_all = urm_all

### Step 1: Split the data and create the evaluator objects

In [42]:
from Evaluation.Evaluator import EvaluatorHoldout

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2965 (22.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2595 (19.9%) Users that have less than 1 test interactions


### Step 2: Define hyperparameter set for the desired model, in this case rp3beta

In [43]:
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {
    "topK": Integer(1, 70),
    "shrink": Real(0, 25)
}

### Step 3: Create SearchBayesianSkopt object, providing the desired recommender class and evaluator objects

In [44]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

recommender_class = ItemKNNCFRecommender

hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                         evaluator_validation=evaluator_validation,
                                         evaluator_test=evaluator_test)

### Step 4: Provide the data needed to create an instance of the model, one trained only on URM_train, the other on URM_train_validation

In [45]:
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
  
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},
    EARLYSTOPPING_KEYWORD_ARGS = {},
)

In [46]:
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train_validation],     # For a CBF model simply put [URM_train_validation, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},
    EARLYSTOPPING_KEYWORD_ARGS = {},
)

### Step 5: Create a result folder and select the number of cases (50 with 30% random is a good number)

In [47]:
import os

output_folder_path = "result_experiments/itemBasedCF"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 10  # using 10 as an example
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

### Step 5: Run!

In [48]:
hyperparameterSearch.search(recommender_input_args,
                       recommender_input_args_last_test = recommender_input_args_last_test,
                       hyperparameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       save_model = "last",
                       output_folder_path = output_folder_path, # Where to save the results
                       output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
                       metric_to_optimize = metric_to_optimize,
                       cutoff_to_optimize = cutoff_to_optimize,
                      )

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 29, 'shrink': 12.705693799599741}
ItemKNNCFRecommender: URM Detected 821 ( 6.3%) users with no interactions.
ItemKNNCFRecommender: URM Detected 481 ( 2.2%) items with no interactions.
Unable to load Cython Compute_Similarity, reverting to Python


Similarity column 22348 (100.0%), 1448.44 column/sec. Elapsed time 15.43 sec
EvaluatorHoldout: Processed 10060 (100.0%) in 9.99 sec. Users per second: 1007
SearchBayesianSkopt: New best config found. Config 0: {'topK': 29, 'shrink': 12.705693799599741} - results: PRECISION: 0.0612028, PRECISION_RECALL_MIN_DEN: 0.1229484, RECALL: 0.1102508, MAP: 0.0279981, MAP_MIN_DEN: 0.0561952, MRR: 0.1846377, NDCG: 0.1039772, F1: 0.0787112, HIT_RATE: 0.4021869, ARHR_ALL_HITS: 0.2261403, NOVELTY: 0.0051989, AVERAGE_POPULARITY: 0.2023570, DIVERSITY_MEAN_INTER_LIST: 0.9767622, DIVERSITY_HERFINDAHL: 0.9976665, COVERAGE_ITEM: 0.3963218, COVERAGE_ITEM_HIT: 0.0717738, ITEMS_IN_GT: 0.7408717, COVERAGE_USER: 0.7723608, COVERAGE_USER_HIT: 0.3106334, USERS_IN_GT: 0.7723608, DIVERSITY_GINI: 0.0747111, SHANNON_ENTROPY: 10.4451116, RATIO_DIVERSITY_HERFINDAHL: 0.9980270, RATIO_DIVERSITY_GINI: 0.2179625, RATIO_SHANNON_ENTROPY: 0.8051735, RATIO_AVERAGE_POPULARITY: 1.7099597, RATIO_NOVELTY: 0.3807417, 

EvaluatorHoldo

### Check the best model

In [49]:
from Recommenders.DataIO import DataIO

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")

search_metadata.keys()

dict_keys(['algorithm_name_recommender', 'algorithm_name_search', 'cutoff_to_optimize', 'exception_list', 'hyperparameters_best', 'hyperparameters_best_index', 'hyperparameters_df', 'metric_to_optimize', 'result_on_earlystopping_df', 'result_on_last', 'result_on_test_best', 'result_on_test_df', 'result_on_validation_best', 'result_on_validation_df', 'time_df', 'time_on_last_df', 'time_on_test_avg', 'time_on_test_total', 'time_on_train_avg', 'time_on_train_total', 'time_on_validation_avg', 'time_on_validation_total'])

In [50]:
hyperparameters_df = search_metadata["hyperparameters_df"]
hyperparameters_df

,topK,shrink
0,29,12.705694
1,40,24.672688
2,6,5.022934
3,70,0.625209
4,19,24.820627
5,48,19.522946
6,70,10.78166
7,1,0.094687
8,70,15.913091
9,1,8.817463


In [51]:
result_on_validation_df = search_metadata["result_on_validation_df"]
result_on_validation_df

,,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
,cutoff,,,,,,,,,,,,,,,,,,,,,
0,10,0.061203,0.122948,0.110251,0.027998,0.056195,0.184638,0.103977,0.078711,0.402187,0.22614,...,0.772361,0.310633,0.772361,0.074711,10.445112,0.998027,0.217963,0.805173,1.70996,0.380742
1,10,0.061064,0.124272,0.111731,0.027905,0.056292,0.18501,0.104349,0.078969,0.403877,0.226067,...,0.772361,0.311939,0.772361,0.052771,9.995276,0.99738,0.153955,0.770497,1.954492,0.368243
2,10,0.059076,0.114776,0.102453,0.027073,0.053919,0.181106,0.099671,0.07494,0.389662,0.220065,...,0.772361,0.30096,0.772361,0.118596,11.086766,0.99875,0.345991,0.854636,1.211342,0.409364
3,10,0.053817,0.10684,0.095782,0.024412,0.048527,0.164116,0.090778,0.068914,0.362425,0.199216,...,0.772361,0.279923,0.772361,0.159063,11.351453,0.998256,0.464051,0.87504,1.493188,0.432408
4,10,0.060338,0.121881,0.109421,0.02779,0.056415,0.186297,0.103799,0.077784,0.400994,0.226211,...,0.772361,0.309712,0.772361,0.057122,10.190333,0.997957,0.166648,0.785534,1.742674,0.374033
5,10,0.061372,0.124427,0.111847,0.02786,0.056161,0.184954,0.104299,0.079255,0.405765,0.225775,...,0.772361,0.313397,0.772361,0.056999,10.052213,0.997329,0.166289,0.774886,1.959121,0.369907
6,10,0.060517,0.122172,0.109917,0.027522,0.055968,0.184935,0.103523,0.078058,0.404672,0.22439,...,0.772361,0.312553,0.772361,0.072454,10.219043,0.997124,0.211378,0.787747,1.981655,0.375253
7,10,0.041302,0.074105,0.065582,0.018021,0.036204,0.131982,0.068332,0.050684,0.300596,0.153667,...,0.772361,0.232169,0.772361,0.076127,9.442331,0.98847,0.222093,0.727873,0.805957,0.448732
8,10,0.060567,0.122737,0.110437,0.027617,0.056126,0.185789,0.10386,0.07823,0.405368,0.225302,...,0.772361,0.31309,0.772361,0.059313,10.010107,0.996925,0.173041,0.771641,2.054025,0.369261


In [52]:
result_best_on_test = search_metadata["result_on_last"]
result_best_on_test

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.084765,0.147568,0.123814,0.043055,0.073056,0.245154,0.133616,0.100634,0.479674,0.320716,...,0.800768,0.384107,0.800768,0.069586,10.413855,0.998083,0.200543,0.80215,1.678646,0.302769


In [53]:
best_hyperparameters = search_metadata["hyperparameters_best"]
best_hyperparameters

{'topK': 29, 'shrink': 12.705693799599741}

In [54]:
time_df = search_metadata["time_df"]
time_df

,train,validation,test
0,15.466024,9.999763,10.222719
1,15.691022,10.648482,NaN
2,14.197238,9.139916,NaN
3,17.944624,10.32967,NaN
4,16.119162,10.12214,NaN
5,18.72022,11.179429,NaN
6,18.365956,11.296324,NaN
7,12.877626,8.450758,NaN
8,15.559988,10.132651,NaN
9,10.944769,8.106719,NaN


In [55]:
exception_list = search_metadata["exception_list"]
exception_list

[None, None, None, None, None, None, None, None, None, None]

# Some useful evaluation

In [31]:
def precision(recommended_items, relevant_items):
    
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    
    precision_score = np.sum(is_relevant, dtype=np.float32) / len(is_relevant)
    
    return precision_score

def recall(recommended_items, relevant_items):
    
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    
    recall_score = np.sum(is_relevant, dtype=np.float32) / relevant_items.shape[0]
    
    return recall_score

def AP(recommended_items, relevant_items):

    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    
    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))
    
    ap_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return ap_score

# We pass as paramether the recommender class and the URM test

def evaluate_algorithm(URM_test, recommender_object, at=5):
    
    cumulative_precision = 0.0
    cumulative_recall = 0.0
    cumulative_AP = 0.0
    
    num_eval = 0


    for user_id in range(URM_test.shape[0]):

        relevant_items = URM_test.indices[URM_test.indptr[user_id]:URM_test.indptr[user_id+1]]
        
        if len(relevant_items)>0:
            
            recommended_items = recommender_object.recommend(user_id, cutoff = 10)
            num_eval+=1

            cumulative_precision += precision(recommended_items, relevant_items)
            cumulative_recall += recall(recommended_items, relevant_items)
            cumulative_AP += AP(recommended_items, relevant_items)
            
    cumulative_precision /= num_eval
    cumulative_recall /= num_eval
    MAP = cumulative_AP / num_eval
    
    print("Recommender results are: Precision = {:.4f}, Recall = {:.4f}, MAP = {:.4f}".format(
        cumulative_precision, cumulative_recall, MAP)) 


First our found model with Bayesian Search:

In [56]:
recommender = ItemKNNCFRecommender(URM_train)
recommender.fit(topK=29,shrink=12.705693799599741)

evaluate_algorithm(URM_test,recommender)

ItemKNNCFRecommender: URM Detected 821 ( 6.3%) users with no interactions.
ItemKNNCFRecommender: URM Detected 481 ( 2.2%) items with no interactions.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 22348 (100.0%), 1228.31 column/sec. Elapsed time 18.19 sec
Recommender results are: Precision = 0.0721, Recall = 0.1064, MAP = 0.0601


In [35]:
recommender = ItemKNNCFRecommender(URM_train)
recommender.fit(topK=14,shrink=7)

evaluate_algorithm(URM_test,recommender)

ItemKNNCFRecommender: URM Detected 827 ( 6.3%) users with no interactions.
ItemKNNCFRecommender: URM Detected 473 ( 2.1%) items with no interactions.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 22348 (100.0%), 1304.59 column/sec. Elapsed time 17.13 sec
Recommender results are: Precision = 0.0746, Recall = 0.1097, MAP = 0.0609
